#### 사용자 정의 데이터셋
- Pytorch에서 딥러닝 시 대량의 데이터 사용에 따른 부하(H/W, S/W) 및 소요 시간 단축
- 대량 데이터셋 전용 처리 모듈 제공
- Dataset과 DataLoader
    * Dataset => 데이터셋 전처리, 텐서화 등의 작업 진행
    * DataLoader => Dataset 인스턴스를 사용해서 batch크기만큼 데이터를 추출

[1] 모듈 로딩 및 데이터 준비 <hr>

In [96]:
# 모듈로딩
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import pandas as pd
from sklearn.preprocessing import LabelEncoder # 타겟 컬럼 수치화

In [97]:
# 데이터
DATA_FILE= '../data/iris.csv'

In [98]:
# CSV => DataFrame
irisDF=pd.read_csv(DATA_FILE)
irisDF.head(2)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa


In [99]:
# 타겟 컬럼 수치화 => LabelEncoder
encoder=LabelEncoder()
encoder.fit(irisDF['variety'])
irisDF['variety']=encoder.transform(irisDF['variety'])

[2] 사용자 정의 데이터셋 클래스 생성

In [100]:
# -------------------------------------------------------------------------
# 클래스목적 : 학습용 데이터셋 텐서화 및 전처리
# 클래스이름 : customDataSet
# 부모클래스 : torch.utils.data.Dataset
# 매개변수 : featureDF, targetDF
# -------------------------------------------------------------------------

In [101]:
class CustomDataset(Dataset):
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드
    def __init__(self, featureDF, targetDF):
        super().__init__()
        self.featureDF=featureDF
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]

    # 데이터의 개수 반환 메서드
    def __len__(self):
        return self.n_rows
    
    # 특정 index의 데이터와 타겟 반환 메서드 => Tensor 반환
    def __getitem__(self, idx):
        featureTS=torch.FloatTensor(self.featureDF.iloc[idx].values)
        targetTS=torch.FloatTensor(self.targetDF.iloc[idx].values)
        return featureTS, targetTS

In [102]:
#------------------------------------------------------------------------------------
# 함수기능 : 파일 확장자별 데이터프레임 변환 기능
# 함수이름 : convertDataFrame
# 매개변수 : data_path
# 함수결과 : DataFrame
#------------------------------------------------------------------------------------
def convertDataFrame(data_path, exit_header=0):
    ext=data_path.rsplit('.')[-1]
    if ext=='csv':
        return pd.read_csv(data_path, header=exit_header)
    elif ext=='json':
        return pd.read_json(data_path, header=exit_header)
    elif ext in ['xlsx','xls']:
        return pd.read_excel(data_path, header=exit_header)
    else : return pd.read_table(data_path, header=exit_header)

In [103]:
#------------------------------------------------------------------------------------
# 클래스기능 : 파일기반 데이터셋
# 클래스이름 : FileDataset
# 매개변수 : data_path 파일 경로
# 부모클래스 : utils.data.Dataset
#------------------------------------------------------------------------------------
class FileDataset(Dataset):
    # 데이터 로딩 및 전처리 진행과 인스턴스 생성 메서드
    def __init__(self, data_path):
        super().__init__()
        # 데이터파일 => DataFrame으로 변환
        dataDF=convertDataFrame(data_path)
        # DataFrame => 피쳐와 타겟 추출
        self.featuresDF=dataDF[dataDF.columns[:-1]]
        self.targetDF=dataDF[dataDF.columns[-1:]]

        self.n_features=self.dataDF.shape[1]
        self.n_rows=self.featuresDF.shape[0]
    
    # 데이터의 개수 반환 메서드
    def __len__(self):
        return self.n_rows
    
    # 특정 index의 데이터와 타겟 반환 메서드 => Tensor 반환
    def __getitem__(self, idx):
        featureTS=torch.FloatTensor(self.featureDF.iloc[idx].values)
        targetTS=torch.FloatTensor(self.targetDF.iloc[idx].values)
        return featureTS, targetTS

[3] 데이터셋 인스턴스 생성 <hr>

In [104]:
featureDF, targetDF=irisDF[irisDF.columns[:-1]], irisDF[irisDF.columns[-1:]]
print(f'featureDF => {featureDF.shape}, targetDF => {targetDF.shape}')

featureDF => (150, 4), targetDF => (150, 1)


In [105]:
# IRIS 데이터셋 인스턴스 생성
irisDS=CustomDataset(featureDF, targetDF)

In [106]:
# IRIS 데이터셋 속성
irisDS.n_features, irisDS.n_rows

(4, 150)

In [107]:
# IRIS 데이터셋 메서드
irisDS[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0.]))

[4] 데이터로더 인스턴스 생성

In [108]:
# 필요한 것 : Dataset 인스턴스, Batch_size
irisDL=DataLoader(irisDS, batch_size=3)

In [109]:
for dataTS, targetTS in irisDL:
    print(dataTS.shape, targetTS.shape)
    break

torch.Size([3, 4]) torch.Size([3, 1])
